# COGS 108 - Project 

In [1]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data = pd.read_csv('Crime_Data.csv')
data.Charge_Description_Orig.unique().size

728

In [3]:
data.head(5)

,agency,Charge_Description_Orig,activityDate,BLOCK_ADDRESS,ZipCode,community
0,SAN DIEGO,GRAND THEFT FROM PERSON,1/3/2018 16:30,7800 BLOCK STALMER STREET,92111.0,SAN DIEGO
1,HARBOR POLICE,SELL LIQUOR TO MINOR (M),9/23/2017 18:28,3200 BLOCK NORTH HARBOR DRIVE,92101.0,SAN DIEGO
2,HARBOR POLICE,"DRUNK IN PUBLIC: ALCOHOL, DRUGS, COMBO OR TOLU...",10/6/2017 8:48,600 BLOCK CONVENTION WAY,92101.0,SAN DIEGO
3,HARBOR POLICE,"DRUNK IN PUBLIC: ALCOHOL, DRUGS, COMBO OR TOLU...",10/11/2017 19:45,3600 BLOCK NORTH HARBOR DRIVE,92101.0,SAN DIEGO
4,HARBOR POLICE,POSS NARCOTIC CONTROLLED SUBS (M),10/21/2017 23:36,100 W BLOCK HARBOR DRIVE,92101.0,SAN DIEGO


In [4]:
data.shape

(60193, 6)

In [5]:
data.columns = ["agency", "crime", "time", "address", "zipcode", "community"]

In [6]:
data.isnull().sum()

agency          0
crime           7
time            0
address         0
zipcode      1228
community    1592
dtype: int64

In [7]:
rows_to_drop = data[data.isnull().any(axis=1)]

In [8]:
rows_to_drop.shape

(2005, 6)

In [9]:
data.dropna(inplace=True)

In [10]:
data.shape

(58188, 6)

In [11]:
# This function takes the crime in a row and dtermines if the suspect was male or female, and adds 
# correlating gender to appropriate row

In [12]:
def whatgender(crime):
    crime = crime.lower()
    
    if "(m)" in crime:
        output = 'male'
    elif "(f)" in crime:
        output = 'female'
    else:
        output = 'nogenderfound'
    return output

In [33]:
data['gender'] = data['crime'].apply(whatgender)
data

,agency,crime,time,address,zipcode,community,gender,realcrime
0,SAN DIEGO,GRAND THEFT FROM PERSON,1/3/2018 16:30,7800 BLOCK STALMER STREET,92111.0,SAN DIEGO,nogenderfound,grand theft from person
1,HARBOR POLICE,SELL LIQUOR TO MINOR (M),9/23/2017 18:28,3200 BLOCK NORTH HARBOR DRIVE,92101.0,SAN DIEGO,male,sell liquor to minor
2,HARBOR POLICE,"DRUNK IN PUBLIC: ALCOHOL, DRUGS, COMBO OR TOLU...",10/6/2017 8:48,600 BLOCK CONVENTION WAY,92101.0,SAN DIEGO,male,"drunk in public: alcohol, drugs, combo or tolu..."
3,HARBOR POLICE,"DRUNK IN PUBLIC: ALCOHOL, DRUGS, COMBO OR TOLU...",10/11/2017 19:45,3600 BLOCK NORTH HARBOR DRIVE,92101.0,SAN DIEGO,male,"drunk in public: alcohol, drugs, combo or tolu..."
4,HARBOR POLICE,POSS NARCOTIC CONTROLLED SUBS (M),10/21/2017 23:36,100 W BLOCK HARBOR DRIVE,92101.0,SAN DIEGO,male,poss narcotic controlled subs
5,SAN DIEGO,POSSESS MARIJUANA 1 OZ OR LESS WHILE DRIVING,1/8/2018 12:25,7600 BLOCK COPLEY PARK PLACE,92111.0,SAN DIEGO,nogenderfound,possess marijuana 1 oz or less while driving
6,HARBOR POLICE,MANUFACTURE/SALE/POSSESS/ETC METAL KNUCKLES (M),11/1/2017 17:15,3600 BLOCK NORTH HARBOR DRIVE,92101.0,SAN DIEGO,male,manufacture/sale/possess/etc metal knuckles
7,HARBOR POLICE,MINOR:KNOWINGLY OPER VEH W/CARRYING ALC (M),11/4/2017 4:30,W LAUREL STREET / PACIFIC HIGHWAY,92101.0,SAN DIEGO,male,minor:knowingly oper veh w/carrying alc
8,HARBOR POLICE,USE/UNDER INFL OF CONTROLLED SUBS (M),11/12/2017 8:11,3100 BLOCK PACIFIC HIGHWAY,92101.0,SAN DIEGO,male,use/under infl of controlled subs
9,HARBOR POLICE,BURGLARY (COMMERCIAL) (F),9/1/2017 19:53,3700 BLOCK NORTH HARBOR DRIVE,92101.0,SAN DIEGO,female,burglary (commercial)


In [40]:
data[data['crime'].str.contains('BURGLARY')]

,agency,crime,time,address,zipcode,community,gender,realcrime
9,HARBOR POLICE,BURGLARY (COMMERCIAL) (F),9/1/2017 19:53,3700 BLOCK NORTH HARBOR DRIVE,92101.0,SAN DIEGO,female,burglary (commercial)
48,SHERIFF,BURGLARY (RESIDENTIAL) (F),11/23/2017 21:30,10000 BLOCK CHAVACAN LANE,91977.0,SPRING VALLEY,female,burglary (residential)
66,SHERIFF,BURGLARY (VEHICLE) (F),2/3/2018 22:30,13800 BLOCK CAM CANADA,92021.0,EL CAJON UNINC,female,burglary (vehicle)
70,CORONADO,BURGLARY/SECOND DEGREE,10/7/2017 11:30,1100 BLOCK G AVENUE,92118.0,CORONADO,nogenderfound,burglary/second degree
103,SHERIFF,BURGLARY (VEHICLE) (F),2/4/2018 17:00,CLAUDIA AVENUE / COTTONWOOD AVENUE,92071.0,SANTEE,female,burglary (vehicle)
105,SHERIFF,BURGLARY (COMMERCIAL) (F),2/4/2018 19:31,9600 BLOCK MISSION GORGE ROAD,92071.0,SANTEE,female,burglary (commercial)
129,EL CAJON,BURGLARY (COMMERCIAL) (F),11/25/2017 10:30,1000 BLOCK PALM AVENUE,92020.0,EL CAJON,female,burglary (commercial)
157,EL CAJON,BURGLARY (COMMERCIAL) (F),12/6/2017 0:25,400 W BLOCK CHASE AVENUE,92020.0,EL CAJON,female,burglary (commercial)
164,HARBOR POLICE,BURGLARY (COMMERCIAL) (F),11/27/2017 1:10,3300 BLOCK ADMIRAL BOLAND WAY,92101.0,SAN DIEGO,female,burglary (commercial)
167,SHERIFF,BURGLARY (RESIDENTIAL) (F),1/16/2018 9:00,2200 BLOCK HELIX STREET,91977.0,SPRING VALLEY,female,burglary (residential)


In [14]:
data['gender'].str.contains('nogenderfound').sum()

22031

In [15]:
genderdata = data

In [16]:
genderdata = genderdata[genderdata.gender != 'nogenderfound']

#This new genderdata DataFrame contains only Male and Female crimes

In [18]:
#This function removes gender from a crime's description

def removegender(crime):
    crime = crime.lower()
    
    if "(m)" in crime:
        output = crime[:-3]
    elif "(f)" in crime:
        output = crime[:-3]
    else:
        output = crime
    return output

data['realcrime'] = data['crime'].apply(removegender)

In [19]:
data['realcrime'].unique().size

695

In [20]:
data['crime'].unique().size

713

In [41]:
data[data['realcrime'].str.contains('attempt')]

,agency,crime,time,address,zipcode,community,gender,realcrime
120,CORONADO,ATTEMPT OPERATE VEHICLE W/O OWNER'S CONSENT,12/2/2017 20:37,1100 BLOCK ISABELLA AVENUE,92118.0,CORONADO,nogenderfound,attempt operate vehicle w/o owner's consent
557,CORONADO,ATTEMPTED PETTY THEFT,12/25/2017 6:53,700 BLOCK G AVENUE,92118.0,CORONADO,nogenderfound,attempted petty theft
861,EL CAJON,ATTEMPTED MURDER (F),10/21/2017 13:40,1100 BLOCK FESLER STREET,92020.0,EL CAJON,female,attempted murder
1033,HARBOR POLICE,ATTEMPTED MURDER (F),10/28/2017 16:51,3300 BLOCK ADMIRAL BOLAND WAY,92101.0,SAN DIEGO,female,attempted murder
2313,OCEANSIDE,ATTEMPTED MURDER (F),10/15/2017 9:13,2800 BLOCK CEDAR ROAD,92056.0,OCEANSIDE,female,attempted murder
2458,SAN DIEGO,ATTEMPT OPERATE VEHICLE W/O OWNER'S CONSENT,12/4/2017 16:00,4100 BLOCK NORMAL STREET,92103.0,SAN DIEGO,nogenderfound,attempt operate vehicle w/o owner's consent
2798,SAN DIEGO,ATTEMPTED MURDER/DEGREE UNSPECIFIED,12/20/2017 10:16,3700 BLOCK VAN DYKE AVENUE,92105.0,SAN DIEGO,nogenderfound,attempted murder/degree unspecified
3061,OCEANSIDE,ATTEMPTED ROBBERY,11/13/2017 18:48,100 S BLOCK PACIFIC STREET,92054.0,OCEANSIDE,nogenderfound,attempted robbery
3690,ESCONDIDO,ATTEMPTED MURDER (F),10/8/2017 18:00,800 E BLOCK 02ND AVENUE,92025.0,ESCONDIDO,female,attempted murder
3724,SAN DIEGO,ATTEMPTED BURGLARY:DEGREE UNSPECIFIED,12/17/2017 6:30,3900 BLOCK FLORIDA STREET,92104.0,SAN DIEGO,nogenderfound,attempted burglary:degree unspecified
